In [ ]:
import json
import matplotlib.pyplot as plt

import bluepyopt as bpopt
import bluepyopt.ephys as ephys

import model
import evaluator
import time
import neuron

import sys

from pathlib import Path
import os

%matplotlib notebook

In [ ]:
#!rm -r x86_64
#!nrnivmodl mechanisms

In [ ]:
probe_type = "linear"
model_name = 'hay' # "hallerman"
model_folder = (Path(".") / f"{model_name}_model").absolute()

In [ ]:
# compile model-specific mechanisms
if not (model_folder / "x86_64").is_dir():
    curr_dir = Path(".")
    os.chdir(model_folder)
    print(os.getcwd())
    os.system("nrnivmodl mechanisms")
    os.chdir(curr_dir)
else:
    print(f"Mechanisms already compiled for {model_name}")

In [ ]:
_ = neuron.load_mechanisms(str(model_folder))

In [ ]:
cell = model.create(model=model_name, morph_modifier="", release=True)

probe = model.define_electrode(cell=cell.LFPyCell, probe_type="linear")

param_names = [param.name for param in cell.params.values() if not param.frozen]

fitness_protocols = evaluator.define_protocols(
    model_name,
    feature_set="multiple",
    feature_file="hay_model/features_list.json",
    electrode=probe,
    protocols_with_lfp=["Step1"]
)

fitness_calculator = ephys.objectivescalculators.ObjectivesCalculator([])

sim = ephys.simulators.LFPySimulator(cell, cvode_active=True, electrode=probe)

eva = ephys.evaluators.CellEvaluator(                    
        cell_model=cell,                                                       
        param_names=param_names,                                                    
        fitness_protocols=fitness_protocols,                                        
        fitness_calculator=fitness_calculator,                                      
        sim=sim)

In [ ]:
fitness_protocols

In [ ]:
t_start = time.time()
responses = eva.run_protocols(protocols=fitness_protocols.values(), param_values={})
print(f"Elapsed time: {time.time() - t_start}")

In [ ]:
fig, axes = plt.subplots(len(responses), figsize=(10, 20), squeeze=False)
for index, (resp_name, response) in enumerate(sorted(responses.items())):
    axes[index, 0].plot(response['time'], response['voltage'], label=resp_name)
    axes[index, 0].set_title(resp_name)
fig.tight_layout()
fig.show()

In [ ]:
responses.keys() #["Step1"]

In [ ]:
plt.figure()
plt.plot(responses['Step1.MEA.LFP']["time"], responses['Step1.MEA.LFP']["voltage"].T)

In [ ]:
responses['Step1.MEA.LFP']["voltage"]

In [ ]:
responses['Step1.MEA.LFP']["voltage"]